In [8]:
%pip install pandas scikit-learn Pillow

  Using cached pillow-11.2.1-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (8.9 kB)
Using cached pillow-11.2.1-cp312-cp312-manylinux_2_28_x86_64.whl (4.6 MB)
Note: you may need to restart the kernel to use updated packages.


In [1]:

import pandas as pd
import os
import shutil
from sklearn.model_selection import train_test_split
import glob
import sys
from PIL import Image

# Definir caminhos
DATA_PATH = 'data/'
ARCHIVE_PATH = 'archive/' 

# Definir proporções de divisão
TRAIN_RATIO = 0.8
VAL_RATIO = 0.1
TEST_RATIO = 0.1 # A soma deve ser 1.0

# Definir tamanho alvo da imagem
TARGET_WIDTH = 256  
TARGET_HEIGHT = 256 

# Definir mapeamentos de diretório (inverso do mapeamento)
VIEW_POSITION_DIR_MAP = {0: 'PA', 1: 'AP'}
FINDING_LABELS_DIR_MAP = {0: '0', 1: '1'} # Usando '0' e '1' como nomes de pastas

In [2]:
print("Iniciando o processo de cópia e organização de imagens com redimensionamento...")
print(f"As imagens serão redimensionadas para {TARGET_WIDTH}x{TARGET_HEIGHT} pixels.")

# --- Carregar os dados processados ---
data_entry_filepath = os.path.join(DATA_PATH, 'data_entry.csv')

if not os.path.exists(data_entry_filepath):
    print(f"Erro: Arquivo de dados processados não encontrado em '{data_entry_filepath}'.")
    print("Por favor, execute o notebook data_extraction.ipynb primeiro para gerar este arquivo.")
    sys.exit(1)

print(f"Carregando dados processados de '{data_entry_filepath}'...")
try:
    processed_df = pd.read_csv(data_entry_filepath)
    print(f"Carregadas {len(processed_df)} entradas.")
    print("Colunas:", processed_df.columns.tolist())
    print("Contagem de valores para 'Finding Labels':\n", processed_df['Finding Labels'].value_counts())
    print("Contagem de valores para 'View Position':\n", processed_df['View Position'].value_counts())
except Exception as e:
    print(f"Erro ao carregar dados processados: {e}")
    sys.exit(1)


Iniciando o processo de cópia e organização de imagens com redimensionamento...
As imagens serão redimensionadas para 256x256 pixels.
Carregando dados processados de 'data/data_entry.csv'...
Carregadas 26634 entradas.
Colunas: ['Image Index', 'Finding Labels', 'Patient Age', 'Patient Gender', 'View Position']
Contagem de valores para 'Finding Labels':
 Finding Labels
1    13317
0    13317
Name: count, dtype: int64
Contagem de valores para 'View Position':
 View Position
1    13456
0    13178
Name: count, dtype: int64


In [3]:
# --- Construir um mapa de 'Image Index' para o caminho da fonte original ---
print(f"\nEscaneando '{ARCHIVE_PATH}' em busca de arquivos de imagem...")
image_path_map = {}
image_count = 0
# Escanear diretórios images_001 a images_012
for i in range(1, 13):
    image_dir = os.path.join(ARCHIVE_PATH, f'images_{i:03d}', 'images')
    if os.path.exists(image_dir):
        # Usar glob para listagem eficiente de arquivos
        files_in_dir = glob.glob(os.path.join(image_dir, '*.png'))
        for f in files_in_dir:
             image_path_map[os.path.basename(f)] = f
             image_count += 1
        # print(f"Escaneado {image_dir} ({len(files_in_dir)} arquivos encontrados). Total de arquivos mapeados: {image_count}") # Escaneamento detalhado
    else:
        print(f"Aviso: Diretório '{image_dir}' não encontrado. Ignorando.")


print(f"Escaneamento finalizado. Total de arquivos de imagem únicos encontrados no arquivo: {len(image_path_map)}")

# Adicionar o caminho da fonte ao dataframe
processed_df['source_path'] = processed_df['Image Index'].map(image_path_map)

# Verificar imagens no dataframe, mas não encontradas no arquivo (não deve haver nenhuma se o notebook for executado corretamente)
missing_images = processed_df[processed_df['source_path'].isnull()]
if not missing_images.empty:
    print(f"\nAviso: {len(missing_images)} imagens listadas em '{data_entry_filepath}' não foram encontradas no arquivo:")
    print(missing_images['Image Index'].tolist()[:10], "...") # Imprimir as 10 primeiras
    # descartar as ausentes para este script
    processed_df.dropna(subset=['source_path'], inplace=True)
    print(f"Removidas {len(missing_images)} entradas devido a arquivos de imagem ausentes. Entradas restantes: {len(processed_df)}")




Escaneando 'archive/' em busca de arquivos de imagem...
Escaneamento finalizado. Total de arquivos de imagem únicos encontrados no arquivo: 112120


In [4]:
# --- Dividir os dados em conjuntos de treino, validação e teste ---
print("\nDividindo dados em conjuntos de treino, validação e teste...")

# Usar estratificação para manter a distribuição de 'Finding Labels' e 'View Position'
# Criar uma coluna combinada para estratificação
processed_df['stratify_col'] = processed_df['Finding Labels'].astype(str) + '_' + processed_df['View Position'].astype(str)

# Divisão inicial: Treino e Temp (Validação + Teste)
train_df, temp_df = train_test_split(
    processed_df,
    test_size=(VAL_RATIO + TEST_RATIO), # 0.1 + 0.1 = 0.2
    random_state=42,
    stratify=processed_df['stratify_col']
)

# Segunda divisão: Validação e Teste a partir de Temp
# Calcular o tamanho do teste relativo ao conjunto temporário
relative_test_size = TEST_RATIO / (VAL_RATIO + TEST_RATIO) # 0.1 / 0.2 = 0.5

val_df, test_df = train_test_split(
    temp_df,
    test_size=relative_test_size, # 0.5
    random_state=42,
    stratify=temp_df['stratify_col']
)

# Descartar a coluna de estratificação temporária dos dataframes divididos
train_df = train_df.drop(columns='stratify_col')
val_df = val_df.drop(columns='stratify_col')
test_df = test_df.drop(columns='stratify_col')

print(f"Tamanho do conjunto de treino: {len(train_df)}")
print(f"Tamanho do conjunto de validação: {len(val_df)}")
print(f"Tamanho do conjunto de teste: {len(test_df)}")

# Verificar divisões
print("\nDistribuição de 'Finding Labels' no conjunto de treino:\n", train_df['Finding Labels'].value_counts(normalize=True))
print("Distribuição de 'Finding Labels' no conjunto de validação:\n", val_df['Finding Labels'].value_counts(normalize=True))
print("Distribuição de 'Finding Labels' no conjunto de teste:\n", test_df['Finding Labels'].value_counts(normalize=True))



Dividindo dados em conjuntos de treino, validação e teste...
Tamanho do conjunto de treino: 21307
Tamanho do conjunto de validação: 2663
Tamanho do conjunto de teste: 2664

Distribuição de 'Finding Labels' no conjunto de treino:
 Finding Labels
1    0.500023
0    0.499977
Name: proportion, dtype: float64
Distribuição de 'Finding Labels' no conjunto de validação:
 Finding Labels
0    0.500188
1    0.499812
Name: proportion, dtype: float64
Distribuição de 'Finding Labels' no conjunto de teste:
 Finding Labels
1    0.5
0    0.5
Name: proportion, dtype: float64


In [5]:
# --- Criar diretórios de destino ---
print("\nCriando diretórios de destino...")

split_dirs = {'train': train_df, 'validation': val_df, 'test': test_df}

for split_name in split_dirs.keys():
    for view_pos_val, view_pos_str in VIEW_POSITION_DIR_MAP.items():
        for finding_label_val, finding_label_str in FINDING_LABELS_DIR_MAP.items():
            dest_dir = os.path.join(DATA_PATH, split_name, view_pos_str, finding_label_str)
            os.makedirs(dest_dir, exist_ok=True)
            print(f"Diretório criado/garantido: {dest_dir}") # Descomentar para criação detalhada

print("Estrutura de diretórios de destino criada.")


Criando diretórios de destino...
Diretório criado/garantido: data/train/PA/0
Diretório criado/garantido: data/train/PA/1
Diretório criado/garantido: data/train/AP/0
Diretório criado/garantido: data/train/AP/1
Diretório criado/garantido: data/validation/PA/0
Diretório criado/garantido: data/validation/PA/1
Diretório criado/garantido: data/validation/AP/0
Diretório criado/garantido: data/validation/AP/1
Diretório criado/garantido: data/test/PA/0
Diretório criado/garantido: data/test/PA/1
Diretório criado/garantido: data/test/AP/0
Diretório criado/garantido: data/test/AP/1
Estrutura de diretórios de destino criada.


In [6]:
# --- Redimensionar e copiar imagens para a nova estrutura ---
print("\nRedimensionando e copiando imagens para a nova estrutura...")
print("Este processo pode levar algum tempo dependendo do número de imagens e do tamanho alvo.")

processed_count = 0
total_files_to_process = len(processed_df)

for split_name, df in split_dirs.items():
    print(f"Processando {len(df)} arquivos para a divisão: {split_name}")
    for index, row in df.iterrows():
        image_index = row['Image Index']
        source_path = row['source_path']
        finding_label_val = row['Finding Labels']
        view_pos_val = row['View Position']

        view_pos_str = VIEW_POSITION_DIR_MAP[view_pos_val]
        finding_label_str = FINDING_LABELS_DIR_MAP[finding_label_val] # Será '0' ou '1'

        dest_dir = os.path.join(DATA_PATH, split_name, view_pos_str, finding_label_str)
        dest_path = os.path.join(dest_dir, image_index) # Manter o nome do arquivo original

        # Realizar redimensionamento e salvamento
        try:
            with Image.open(source_path) as img:
                # Redimensionar a imagem usando o filtro LANCZOS para uma boa qualidade de redução
                resized_img = img.resize((TARGET_WIDTH, TARGET_HEIGHT), Image.Resampling.LANCZOS)

                # Salvar a imagem redimensionada no caminho de destino
                resized_img.save(dest_path, format='PNG')

            processed_count += 1
            if processed_count % 500 == 0: # Imprimir progresso a cada 500 arquivos
                print(f"  Processados {processed_count}/{total_files_to_process} arquivos...")

        except FileNotFoundError:
             print(f"  Erro: Arquivo fonte não encontrado para {image_index} em {source_path}. Ignorando.")
        except Exception as e:
             print(f"  Erro ao processar {image_index} de {source_path} para {dest_path}: {e}")


print(f"\nProcessamento e cópia de imagens completos. Total de arquivos processados: {processed_count}")
print("Processo finalizado.")


Redimensionando e copiando imagens para a nova estrutura...
Este processo pode levar algum tempo dependendo do número de imagens e do tamanho alvo.
Processando 21307 arquivos para a divisão: train
  Processados 500/26634 arquivos...
  Processados 1000/26634 arquivos...
  Processados 1500/26634 arquivos...
  Processados 2000/26634 arquivos...
  Processados 2500/26634 arquivos...
  Processados 3000/26634 arquivos...
  Processados 3500/26634 arquivos...
  Processados 4000/26634 arquivos...
  Processados 4500/26634 arquivos...
  Processados 5000/26634 arquivos...
  Processados 5500/26634 arquivos...
  Processados 6000/26634 arquivos...
  Processados 6500/26634 arquivos...
  Processados 7000/26634 arquivos...
  Processados 7500/26634 arquivos...
  Processados 8000/26634 arquivos...
  Processados 8500/26634 arquivos...
  Processados 9000/26634 arquivos...
  Processados 9500/26634 arquivos...
  Processados 10000/26634 arquivos...
  Processados 10500/26634 arquivos...
  Processados 11000/2663